In [4]:
import folium
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits
from mpl_toolkits.basemap import Basemap
import matplotlib
matplotlib.rc('font', family='Verdana')
from geopandas.geoseries import *
from shapely.geometry.polygon import Polygon

%matplotlib inline

#### 1. Загрузите агрегированные данные о поездках в мае 2016. Просуммируйте общее количество поездок такси из каждой географической зоны и посчитайте количество ячеек, из которых в мае не было совершено ни одной поездки.

In [8]:
# Читаем данные
data = pd.read_csv('../data/aggregated/2016-05.csv')
regions = pd.read_csv('../data/regions.csv', sep=';')

In [9]:
# Суммируем поездки из каждой зоны
data['trips'] = np.sum(data.values[:, 1:], axis=1)
trip_count = data.trips.value_counts()

# Количество ячеек, из которых в мае не было совершено ни одной поездки
print('\nКоличество ячеек, из которых в мае не было совершено ни одной поездки = ', trip_count[0])



Количество ячеек, из которых в мае не было совершено ни одной поездки =  1284


#### 2.Нарисуйте статическую карту Нью-Йорка. Поставьте на карте точку там, где находится Эмпайр-Стейт-Билдинг.
#### 3.Поверх статической карты Нью-Йорка визуализируйте данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую вы окрашиваете каждую ячейку, показывала суммарное количество поездок такси из неё.

In [10]:
# Коррдинаты Empire State Building
ESB_lat, ESB_long = 40.747778, -73.985833
 

In [11]:
# Районы города 
lon = np.unique([regions.west, regions.east])
lat = np.unique([regions.south, regions.north])

lon, lat = np.meshgrid(lon, lat)
trips_shape = data.trips.values.reshape(50,50)


In [ ]:
%%time
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 7))
m = Basemap(area_thresh=0.01, ax=ax,
            llcrnrlat= 40.49612, urcrnrlat= 40.91553,
            llcrnrlon=-74.25559, urcrnrlon=-73.70001,
            resolution='f',
            projection='cyl')

# Empire State Building
m.plot(x=ESB_long, y=ESB_lat, marker='o' , color='b', zorder=11)
plt.text(x=ESB_long-0.03, y=ESB_lat+0.01, color='b', s='Empire State Building', zorder=11)

# Землю и воду на задний план, 
m.fillcontinents(color='green', lake_color='aqua', zorder=0, alpha=0, )

# Основная карта-диаграмма
cmap=plt.get_cmap('Paired')
plt.pcolormesh(lon, lat, trips_shape, zorder=2, color='lightgray', cmap=cmap)

# Сбоку рисуем colorbar
plt.imshow(X=trips_shape, aspect='auto', cmap=cmap, alpha=0.3, zorder=12)
plt.clim(0, max(data.trips) ) 
m.colorbar(label='Trips')

# Очертания берегов
m.drawcoastlines(color='gray', zorder=11)

plt.title(u"Исследуемая зона - квадрат со стороной ~46км")
print('\n                                 Голубой цвет - очень мало поездок\n')
plt.show()


#### 4.Вставьте интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставьте метку там, где находится статуя свободы.
#### 5. Нарисуйте на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.

In [12]:
#Нью-Йорк вписан в прямоугольник от -74.25559 до -73.70001 градусов долготы и от 40.49612 до 40.91553 широты. 
NY = np.ravel([-74.25559, -73.70001, 40.49612, 40.91553])

# Координата статуи свободы
liberty = [40.689167, -74.044583]

In [13]:
# Для folium надо подготовить индексы.
# Согласование данных для choropleth, 
# Индекс regions.region в виде "01", "02"....., "10"....

regions.region = ['{:0=2}'.format(s) for s in regions.region]

data['trips_mean'] = (data.trips/744).astype(dtype=np.int64)
data.region = ['{:0=2}'.format(s) for s in data.region]

In [18]:
# GeoSeries содержит квадратные полигоны
ll = [p for p in zip(regions.west, regions.south)]    
lr = [p for p in zip(regions.east, regions.south)]    
ul = [p for p in zip(regions.west, regions.north)]    
ur = [p for p in zip(regions.east, regions.north)]    

poly = GeoSeries([Polygon(p) for p in zip(ll, lr, ur, ul)])
poly.index = ['{:0=2}'.format(s) for s in poly.index+1]

poly_json = poly.to_json()


In [ ]:
# Карта
m = folium.Map(min_lon=NY[0], max_lon=NY[1], min_lat=NY[2], max_lat=NY[3],
               location=liberty, 
               zoom_start=9)

# Маркер для Статуи Свободы
folium.Marker(liberty, popup="Statue of Liberty ").add_to(m)


# Choropleth - интерактивная карта
m.choropleth(geo_str=poly_json, data=data, columns=['region', 'trips_mean'],
             fill_color='Set2', fill_opacity=0.5, 
             line_color='black', line_opacity=.5, line_weight=.1,
             key_on = 'feature.id',
             threshold_scale=[0, 5, 500, max(data.trips_mean)+1],
            )
#m.save(os.path.join('results', 'RectangleMarker_and_PolygonMarker_0.html'))
print('\n                                           Оранжевый цвет - минимум поездок, 0-5 в час\n')
m

In [ ]:
print('оранжевый цвет - очень мало поездок, 0-5 в час')

#### 6.Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайте количество оставшихся. Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.

In [ ]:
poly_05 = poly[data.region[data.trips_mean>=5]]

poly_05_json = poly_05.to_json()

m_05 = folium.Map(min_lon=NY[0], max_lon=NY[1], min_lat=NY[2], max_lat=NY[3],
                  location=liberty,
                  tiles='Stamen Toner',
                  zoom_start=11)


m_05.choropleth(geo_str=poly_05_json, data=data, columns=['region', 'trips_mean'],
                fill_color='Set2', fill_opacity=0.8, 
                line_color='black', line_opacity=.5, line_weight=.1,
                key_on = 'feature.id',
                threshold_scale=[0, 5, 500, max(data.trips_mean)+1],
                legend_name='kjkh')

print('Всего ячеек', poly.shape)
print('Ячейки, из которых в мае совершается в среднем не меньше 5 поездок в час.= ', poly_05.shape[0])

m_05

#### На карте видно, что из любой ячейки, из которой совершается не менне 5 поездок в час, есть возможность проехать на такси.

In [ ]:

len(poly_05.index)

In [ ]:

# Функция сохранения в файл массива
def save_arr(fname, arr, sep=','):
    #print(len(arr), type(arr))
    with open(fname,"w") as fout:
        fout.write(sep.join([el for el in arr]))


In [ ]:
save_arr(fname='../data/regs_05.csv', arr=poly_05.index)
poly_05.head()